# Landscape Singlet Lens

## Landscape rear configuration

In [ ]:
import torch
import torch.nn as nn
import torchlensmaker as tlm
import torch.optim as optim

# Setup two spherical surfaces with initial radiuses
surface1 = tlm.Sphere(diameter=30, r=tlm.parameter(torch.tensor(-55.)))
surface2 = tlm.Sphere(diameter=30, r=tlm.parameter(torch.tensor(-35.)))

lens = tlm.Lens(surface1, surface2, (1.0, 1.5), outer_thickness=3)

# Build the optical sequence
optics = nn.Sequential(
    tlm.ObjectAtInfinity(beam_diameter=10, angular_size=20),
    tlm.Gap(15),
    lens,
    tlm.Gap(120),
    tlm.ImagePlane(diameter=60),
)

tlm.show(optics, dim=2, sampling={"base": 10, "object": 5, "sampler": "uniform"}, color_dim="object")

tlm.plot_magnification(optics, sampling={"base": 10, "object": 5, "sampler": "uniform"})


In [ ]:
# Find the best parameters for the shapes
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    sampling = {"base": 10, "object": 5, "sampler": "uniform"},
    dim = 2,
    num_iter = 200,
).plot()

# TODO always zero out y axis of loss plot

In [ ]:
tlm.show(optics, dim=2, end=200, sampling={"base": 10, "object": 5, "sampler": "uniform"}, color_dim="object")
tlm.plot_magnification(optics, sampling={"base": 10, "object": 5, "sampler": "uniform"})

## Landscape front configuration

In [ ]:
import torch
import torch.nn as nn
import torchlensmaker as tlm
import torch.optim as optim

# Setup two spherical surfaces with initial radiuses
surface1 = tlm.Sphere(diameter=30, r=tlm.parameter(torch.tensor(35.)))
surface2 = tlm.Sphere(diameter=30, r=tlm.parameter(torch.tensor(55.)))

lens = tlm.Lens(surface1, surface2, (1.0, 1.5), outer_thickness=3)

# Build the optical sequence
optics = nn.Sequential(
    tlm.ObjectAtInfinity(beam_diameter=10, angular_size=20),
    tlm.Gap(15),
    lens,
    tlm.Gap(120),
    tlm.ImagePlane(diameter=120),
)

tlm.show(optics, dim=2, end=200, sampling={"base": 10, "object": 5, "sampler": "uniform"}, color_dim="object")

#plot_magnification(optics, sampling={"base": 10, "object": 5, "sampler": "uniform"})

In [ ]:
# Find the best parameters for the shapes
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    sampling = {"base": 10, "object": 5, "sampler": "uniform"},
    dim = 2,
    num_iter = 200,
).plot()

In [ ]:

tlm.show(optics, dim=2, end=200, sampling={"base": 10, "object": 10, "sampler": "uniform"}, color_dim="object")

#plot_magnification(optics, sampling={"base": 10, "object": 5, "sampler": "uniform"})

In [ ]:
from IPython.display import display
import build123d as bd

part = tlm.lens_to_part(lens)
display(part)